# Working with H3 in Vgrid DGGS

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.vgrid.vn/lab/index.html?path=vgrid/01_h3.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeoshub/vgrid/blob/main/docs/notebooks/01_h3.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeoshub/vgrid/HEAD?filepath=docs/notebooks/01_h3.ipynb)
[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeoshub/vgrid/blob/main/docs/notebooks/01_h3.ipynb)
[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.gishub.vn/lab/index.html?path=notebooks/vgrid/01_h3.ipynb)

Full Vgrid DGGS documentation is available at [vgrid document](https://vgrid.gishub.vn).

To work with Vgrid DGGS directly in GeoPandas and Pandas, please use [vgridpandas](https://pypi.org/project/vgridpandas/). Full Vgridpandas DGGS documentation is available at [vgridpandas document](https://vgridpandas.gishub.vn).

To work with Vgrid DGGS in QGIS, install the [Vgrid Plugin](https://plugins.qgis.org/plugins/vgridtools/).

To visualize DGGS in Maplibre GL JS, try the [vgrid-maplibre](https://www.npmjs.com/package/vgrid-maplibre) library.

For an interactive demo, visit the [Vgrid Homepage](https://vgrid.vn).

## Install vgrid

In [ ]:
# %pip install vgrid --upgrade

## latlon2h3

In [ ]:
from vgrid.conversion.latlon2dggs import latlon2h3

lat = 10.775276
lon = 106.706797
res = 9
h3_id = latlon2h3(lat, lon, 10)
h3_id

### H3 to Shapely Polygon

In [ ]:
from vgrid.conversion.dggs2geo.h32geo import h32geo

h3_geo = h32geo(h3_id)
h3_geo

### H3 to GeoJSON

In [ ]:
from vgrid.conversion.dggs2geo.h32geo import h32geojson

h3_geojson = h32geojson(h3_id)
h3_geojson

### Vector to H3

In [ ]:
### H3 to Shapely Polygon
from vgrid.conversion.vector2dggs.vector2h3 import vector2h3

file_path = "https://raw.githubusercontent.com/opengeoshub/vopendata/main/shape/polygon2.geojson"
vector_to_h3 = vector2h3(
    file_path,
    compact=False,
    topology=True,
    resolution=10,
    predicate="intersects",
    output_format="gpd",
)

vector_to_h3.plot(edgecolor="white")

### H3 Compact

In [ ]:
from vgrid.conversion.dggscompact.h3compact import h3compact

h3_compacted = h3compact(vector_to_h3, output_format="gpd")
h3_compacted.plot(edgecolor="white")

### H3 Expand

In [ ]:
from vgrid.conversion.dggscompact.h3compact import h3expand

h3_expanded = h3expand(h3_compacted, resolution=11, output_format="gpd")
h3_expanded.plot(edgecolor="white")

### H3 Binning

In [ ]:
from vgrid.binning.h3bin import h3bin

file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/csv/dist1_pois.csv"
)
stats = "majority"
h3_bin = h3bin(
    file_path,
    resolution=10,
    stats=stats,
    numeric_field="confidence",
    # category="category",
    output_format="gpd",
)
h3_bin.plot(
    column=stats,  # numeric column to base the colors on
    cmap="Spectral_r",  # color scheme (matplotlib colormap)
    legend=True,
    linewidth=0.2,  # boundary width (optional)
)

### Raster to H3

#### Download and open raster

In [ ]:
from vgrid.utils.io import download_file
import rasterio
from rasterio.plot import show

raster_url = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/raster/rgb.tif"
)
raster_file = download_file(raster_url)
src = rasterio.open(raster_file, "r")
print(src.meta)
show(src)

#### Convert raster to H3

In [ ]:
# %pip install folium

In [ ]:
from vgrid.conversion.raster2dggs.raster2h3 import raster2h3

#### Convert to H3
raster_to_h3 = raster2h3(raster_file, output_format="gpd")
raster_to_h3.head()

# Visualize the output
import folium

m = folium.Map(tiles="CartoDB positron", max_zoom=28)

h3_layer = folium.GeoJson(
    raster_to_h3,
    style_function=lambda x: {
        "fillColor": f"rgb({x['properties']['band_1']}, {x['properties']['band_2']}, {x['properties']['band_3']})",
        "fillOpacity": 1,
        "color": "black",
        "weight": 0.5,
    },
    popup=folium.GeoJsonPopup(
        fields=["h3", "resolution", "band_1", "band_2", "band_3", "cell_area"],
        aliases=["H3 ID", "Resolution", "Band 1", "Band 2", "Band 3", "Area (m²)"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(h3_layer.get_bounds())

# Display the map
m

### H3 Generator

In [ ]:
from vgrid.generator.h3grid import h3grid

h3_grid = h3grid(resolution=0)
h3_grid.plot(edgecolor="white")

### H3 Inspect

In [ ]:
from vgrid.stats.h3stats import h3inspect

resolution = 3
h3_inspect = h3inspect(resolution)
h3_inspect.head()

### H3 Normalized Area Histogram

In [ ]:
from vgrid.stats.h3stats import h3_norm_area_hist

h3_norm_area_hist(h3_inspect)

### Distribution of H3 Area Distortions

In [ ]:
from vgrid.stats.h3stats import h3_norm_area

h3_norm_area(h3_inspect)

### H3 IPQ Compactness Histogram

Isoperimetric Inequality (IPQ) Compactness (suggested by Osserman, 1987):
$$C_{IPQ} = \frac{4 \pi A}{p^2}$$
The range of the IPQ compactness metric is [0,1]. 

A circle represents the maximum compactness with a value of 1. 

As shapes become more irregular or elongated, their compactness decreases toward 0.

In [ ]:
from vgrid.stats.h3stats import h3_compactness_hist

h3_compactness_hist(h3_inspect)

### Distribution of H3 IPQ Compactness

In [ ]:
from vgrid.stats.h3stats import h3_compactness

h3_compactness(h3_inspect)

### H3 Statistics

Characteristic Length Scale (CLS - suggested by Ralph Kahn): the diameter of a spherical cap of the same cell's area

In [ ]:
from vgrid.stats.h3stats import h3stats

h3_stats = h3stats("km")
h3_stats